In [1]:
import json
import re
import requests
import shutil
from urllib.parse import urlparse
import os

from tqdm import tqdm

In [2]:
DATA_FOLDER = "data"

urls = dict()
with open(f'{DATA_FOLDER}/followers_rehydrated.jsonl') as i:
    for l in i:
        j = json.loads(l)
        urls[j['id']] = j['profile_image_url']

with open(f'{DATA_FOLDER}/follower_pic_urls.csv', 'w+') as o:
    o.write('\n'.join(f"{k},{v}" for k, v in urls.items()))

In [3]:
len(urls), len(set(urls.values()))

(5516064, 4192342)

In [4]:
from collections import Counter
url_counts = Counter(urls.values())

In [5]:
url_counts.most_common(10) # the default and empty urls should be blacklisted

[('https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png',
  1320808),
 ('', 2916),
 ('https://pbs.twimg.com/profile_images/852969926163808256/SaSysP9k_normal.jpg',
  1),
 ('https://pbs.twimg.com/profile_images/1454018403274948610/9QgH-Lj3_normal.jpg',
  1),
 ('https://pbs.twimg.com/profile_images/640041222082465792/4g-y6Ztk_normal.jpg',
  1),
 ('https://pbs.twimg.com/profile_images/1426330123238744064/9Itp8R6e_normal.jpg',
  1),
 ('https://pbs.twimg.com/profile_images/545913341063864320/8S9obh5d_normal.jpeg',
  1),
 ('https://pbs.twimg.com/profile_images/647018342511869952/xNYCPG6z_normal.jpg',
  1),
 ('https://pbs.twimg.com/profile_images/983837066369863680/ltPhdbYq_normal.jpg',
  1),
 ('https://pbs.twimg.com/profile_images/571637185968939009/Fzu0V3ZR_normal.jpeg',
  1)]

In [ ]:
errors = []
overwrite = True

for k, v in tqdm(urls.items(), desc='pictures', total = len(urls)):
    if v in {"",'https://abs.twimg.com/sticky/default_profile_images/default_profile_normal.png'}:
        continue
    path = urlparse(v).path
    ext = os.path.splitext(path)[1]
    v = re.sub(f"^(?P<needed>.*)(?P<unneeded>_normal)(?P<ext>{ext})$",r"\g<needed>\g<ext>",  v)
    out_path = f"{DATA_FOLDER}/img/{k[:2]}/{k[2:4]}/{k}{ext}"
    if os.path.exists(out_path) and not overwrite:
        continue
    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    try:
        r = requests.get(v, stream=True)
        if r.status_code == 200:
            with open(out_path, 'wb') as f:
                r.raw.decode_content = True
                shutil.copyfileobj(r.raw, f)
        else:
            errors.append(k)
    except:
        errors.append(k)


pictures:   0%|          | 1226/5516064 [07:20<543:31:39,  2.82it/s]

In [12]:
list(urls.values())[:3]

['https://pbs.twimg.com/profile_images/852969926163808256/SaSysP9k_normal.jpg',
 'https://pbs.twimg.com/profile_images/1454018403274948610/9QgH-Lj3_normal.jpg',
 'https://pbs.twimg.com/profile_images/640041222082465792/4g-y6Ztk_normal.jpg']

In [16]:
urlparse('https://pbs.twimg.com/profile_images/640041222082465792/4g-y6Ztk_normal.jpg').path

'/profile_images/640041222082465792/4g-y6Ztk_normal.jpg'

In [17]:
v = 'https://pbs.twimg.com/profile_images/640041222082465792/4g-y6Ztk_normal.jpg'

In [51]:
path = urlparse(v).path
ext = os.path.splitext(path)[1]



'https://pbs.twimg.com/profile_images/640041222082465792/4g-y6Ztk.jpg'

In [50]:
ext

''